<a href="https://colab.research.google.com/github/anastasiiaCher/python-for-DPaV/blob/main/labs/%D0%9B%D0%A06_%D0%9E%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_%D0%B8_%D0%BA%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D1%8F_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%B2%D1%8B%D1%85_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Постановка задачи


**Цель работы:** применение наивного байесовского классификатора для определения тональности текстов на русском языке.

**Краткое описание:** в ходе выполнения лабораторной работы необходимо провести предобработку сырых текстовых данных и настроить гиперпараметры нескольких векторайзеров так, чтобы точность классификации (`accuracy`) наивным байесовским классификатором из `sklearn` составила не менее 0.74.

# Данные


В папке [Data/lab6](https://drive.google.com/drive/folders/1jo_V1RYyFKNa2TR_VNZYMGKUi3T1Jpze?usp=sharing) расположена таблица с русскоязычными текстовыми отзывами на предметы одежды. Также эти же данные можно найти [по сслыке](https://github.com/sismetanin/rureviews). Все задания лабораторной работы необходимо выполнять по этим данным.

##### **Формат данных**

```
review	sentiment
Очень плохо упакован, весь  мятый и смотрится не так как на фото	negative
заказ не пришёл но деньги вернули	neautral
Мягкая удобная тёплая. Все замечательно. Ничего не торчит запахов нет. Продавцу + в карму	positive
...
```
##### **Описание полей**

 - ```review``` — сырой текст отзыва в большинстве случаев на русском языке, встречаются отдельные записи на английском;
 - ```sentiment``` — тональность отзыва (позитивный, нейтральный, негативный), классы сбалансированы.

# Практические задания

## Предварительная обработка данных

### Токенизация

Токенизировать – значит разделить текст на части: слова, ключевые слова, фразы, символы и т.д., иными словами – **токены**.

Самый простой способ токенизировать текст – разделить с помощью метода `split()`. Но `split` упускает очень много всего, например, не отделяет пунктуацию от слов. Кроме этого, есть ещё много менее тривиальных проблем, поэтому лучше использовать готовые токенизаторы, к примеру:

- `Yargy` парсер. Документация: https://github.com/natasha/yargy
- Парсер из модуля `nltk`. Учебник по nltk: https://www.nltk.org/book



In [ ]:
# токенизация через split()

text = '''
Продаётся LADA 4x4. ПТС 01.12.2018, куплена 20 января 19 года, 10 000 км пробега.
Комплектация полная. Новая в салоне 750 000, отдам за 650 000.
Возможен обмен на ВАЗ-2110 или ВАЗ 2109 с вашей доплатой.
Краснодар, ул. Миклухо-Маклая, д. 4/5, подьезд 1
Тел. 8(999)1234567, 8 903 987-65-43, +7 (351) 111 22 33
И.И. Иванов (Иван Иванович)
'''

tokens = text.split()
print(f"split() определил {len(tokens)} токенов")
print(tokens)

split() определил 56 токенов
['Продаётся', 'LADA', '4x4.', 'ПТС', '01.12.2018,', 'куплена', '20', 'января', '19', 'года,', '10', '000', 'км', 'пробега.', 'Комплектация', 'полная.', 'Новая', 'в', 'салоне', '750', '000,', 'отдам', 'за', '650', '000.', 'Возможен', 'обмен', 'на', 'ВАЗ-2110', 'или', 'ВАЗ', '2109', 'с', 'вашей', 'доплатой.', 'Краснодар,', 'ул.', 'Миклухо-Маклая,', 'д.', '4/5,', 'подьезд', '1', 'Тел.', '8(999)1234567,', '8', '903', '987-65-43,', '+7', '(351)', '111', '22', '33', 'И.И.', 'Иванов', '(Иван', 'Иванович)']


In [ ]:
%%capture
# токенизация через yargy
!pip install yargy

In [ ]:
from yargy.tokenizer import MorphTokenizer

tknzr = MorphTokenizer()
tokens = [_.value for _ in tknzr(text)]
print(f"yargy определил {len(tokens)} токенов")
print(tokens)

yargy определил 107 токенов
['\n', 'Продаётся', 'LADA', '4', 'x', '4', '.', 'ПТС', '01', '.', '12', '.', '2018', ',', 'куплена', '20', 'января', '19', 'года', ',', '10', '000', 'км', 'пробега', '.', '\n', 'Комплектация', 'полная', '.', 'Новая', 'в', 'салоне', '750', '000', ',', 'отдам', 'за', '650', '000', '.', '\n', 'Возможен', 'обмен', 'на', 'ВАЗ', '-', '2110', 'или', 'ВАЗ', '2109', 'с', 'вашей', 'доплатой', '.', '\n', 'Краснодар', ',', 'ул', '.', 'Миклухо', '-', 'Маклая', ',', 'д', '.', '4', '/', '5', ',', 'подьезд', '1', '\n', 'Тел', '.', '8', '(', '999', ')', '1234567', ',', '8', '903', '987', '-', '65', '-', '43', ',', '+', '7', '(', '351', ')', '111', '22', '33', '\n', 'И', '.', 'И', '.', 'Иванов', '(', 'Иван', 'Иванович', ')', '\n']


In [ ]:
%%capture
# токенизация через nltk
import nltk

nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
from nltk.tokenize import word_tokenize

tokens = word_tokenize(text)
print(f"nltk определил {len(tokens)} токенов")
print(tokens)

nltk определил 81 токенов
['Продаётся', 'LADA', '4x4', '.', 'ПТС', '01.12.2018', ',', 'куплена', '20', 'января', '19', 'года', ',', '10', '000', 'км', 'пробега', '.', 'Комплектация', 'полная', '.', 'Новая', 'в', 'салоне', '750', '000', ',', 'отдам', 'за', '650', '000', '.', 'Возможен', 'обмен', 'на', 'ВАЗ-2110', 'или', 'ВАЗ', '2109', 'с', 'вашей', 'доплатой', '.', 'Краснодар', ',', 'ул', '.', 'Миклухо-Маклая', ',', 'д', '.', '4/5', ',', 'подьезд', '1', 'Тел', '.', '8', '(', '999', ')', '1234567', ',', '8', '903', '987-65-43', ',', '+7', '(', '351', ')', '111', '22', '33', 'И.И', '.', 'Иванов', '(', 'Иван', 'Иванович', ')']


В `nltk` есть довольно много токенизаторов. Токенизатор подбирается, исходя из языка и  требований задачи.

In [ ]:
from nltk import tokenize

print(f"Количество токенизаторов в nltk: {len(dir(tokenize))}")
dir(tokenize)[:5]

Количество токенизаторов в nltk: 58


['BlanklineTokenizer',
 'LegalitySyllableTokenizer',
 'LineTokenizer',
 'MWETokenizer',
 'NLTKWordTokenizer']

### Очистка текста

Очистка текста включает в себя несколько этапов:
- удаление пунктуации;
- преобразование регистра;
- удаление стоп-слов.

Сюда же включается исправление опечаток и этапы очистки от лишней информации (тегов html-разметки, url, непонятных символов и др.) и приведения отдельных потенциально важных элементов текстов к единому виду (времени или дат, например).

In [ ]:
# набор пунктуационных символов зависит от задачи и текста
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
# удаление стандартной пунктуации
punct = string.punctuation
clean_words = [w.strip(punct) for w in word_tokenize(text)]
print(clean_words)

['Продаётся', 'LADA', '4x4', '', 'ПТС', '01.12.2018', '', 'куплена', '20', 'января', '19', 'года', '', '10', '000', 'км', 'пробега', '', 'Комплектация', 'полная', '', 'Новая', 'в', 'салоне', '750', '000', '', 'отдам', 'за', '650', '000', '', 'Возможен', 'обмен', 'на', 'ВАЗ-2110', 'или', 'ВАЗ', '2109', 'с', 'вашей', 'доплатой', '', 'Краснодар', '', 'ул', '', 'Миклухо-Маклая', '', 'д', '', '4/5', '', 'подьезд', '1', 'Тел', '', '8', '', '999', '', '1234567', '', '8', '903', '987-65-43', '', '7', '', '351', '', '111', '22', '33', 'И.И', '', 'Иванов', '', 'Иван', 'Иванович', '']


In [ ]:
# преобразование регистра
clean_words = [w.lower() for w in clean_words if w != '']
print(clean_words)

['продаётся', 'lada', '4x4', 'птс', '01.12.2018', 'куплена', '20', 'января', '19', 'года', '10', '000', 'км', 'пробега', 'комплектация', 'полная', 'новая', 'в', 'салоне', '750', '000', 'отдам', 'за', '650', '000', 'возможен', 'обмен', 'на', 'ваз-2110', 'или', 'ваз', '2109', 'с', 'вашей', 'доплатой', 'краснодар', 'ул', 'миклухо-маклая', 'д', '4/5', 'подьезд', '1', 'тел', '8', '999', '1234567', '8', '903', '987-65-43', '7', '351', '111', '22', '33', 'и.и', 'иванов', 'иван', 'иванович']


**Стоп-слова** — высокочастотные слова, которые не дают никакой информации о конкретном тексте. Они составляют верхушку частотного списка в любом языке. Набор стоп-слов не универсален, он будет зависеть от задачи. Обычно начинают с готовых списков стоп-слов, а потом их дополняют.

In [ ]:
# список стоп-слов для русского языка в nltk
from nltk.corpus import stopwords
sw = stopwords.words('russian')
print(f"Количество стоп-слов: {len(sw)}")
print(sw)

Количество стоп-слов: 151
['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве',

In [ ]:
# удаление стоп-слов
print([w if w not in sw else print(w) for w in clean_words])

в
за
на
или
с
['продаётся', 'lada', '4x4', 'птс', '01.12.2018', 'куплена', '20', 'января', '19', 'года', '10', '000', 'км', 'пробега', 'комплектация', 'полная', 'новая', None, 'салоне', '750', '000', 'отдам', None, '650', '000', 'возможен', 'обмен', None, 'ваз-2110', None, 'ваз', '2109', None, 'вашей', 'доплатой', 'краснодар', 'ул', 'миклухо-маклая', 'д', '4/5', 'подьезд', '1', 'тел', '8', '999', '1234567', '8', '903', '987-65-43', '7', '351', '111', '22', '33', 'и.и', 'иванов', 'иван', 'иванович']


### Нормализация текста

**Нормализация токенов** (token normalization) – это процесс приведения токенов к канонической форме, чтобы устранить несущественные различия между последовательностями символов.

Для многих задач естественно рассматривать как отдельный признак каждое *слово*, а не каждую его отдельную форму.

**Стемминг** (англ. stemming — находить происхождение) — это процесс нахождения основы слова для заданного исходного слова. Основа слова не обязательно совпадает с морфологическим корнем слова.

**Лемматизация** – это сведение разных форм одного слова к начальной форме – **лемме**.

**Пример:**

Лексема `saw` после стемминга может стать буквой `s`, а в ходе лемматизации либо словом `see`, либо `saw`, в зависимости от того, глагол это или существительное.

#### Стемминг

Стемминг — отсечение от слова окончаний и суффиксов, чтобы оставшаяся часть, называемая stem, была одинаковой для всех грамматических форм слова. Стем необязательно совпадает с морфлогической основой слова. Одинаковый стем может получиться и не у однокоренных слов и наоборот — в этом проблема стемминга.

1-ый вид ошибки: белый, белка, белье $\implies$ бел

2-ой вид ошибки: трудность, трудный $\implies$ трудност, труд

3-ий вид ошибки: быстрый, быстрее $\implies$ быст, побыстрее $\implies$ побыст

Самый простой алгоритм, [алгоритм Портера](https://ru.wikipedia.org/wiki/%D0%A1%D1%82%D0%B5%D0%BC%D0%BC%D0%B5%D1%80_%D0%9F%D0%BE%D1%80%D1%82%D0%B5%D1%80%D0%B0), состоит из 5 циклов команд, на каждом цикле – операция удаления / замены суффикса. Возможны вероятностные расширения алгоритма.

**Snowball stemmer** - улучшенный вариант стеммера Портера; в отличие от него умеет работать не только с английским текстом.

In [ ]:
from nltk.stem.snowball import SnowballStemmer

SnowballStemmer.languages

('arabic',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'hungarian',
 'italian',
 'norwegian',
 'porter',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish')

In [ ]:
poem = '''
По морям, играя, носится
с миноносцем миноносица.
Льнет, как будто к меду осочка,
к миноносцу миноносочка.
И конца б не довелось ему,
благодушью миноносьему.
Вдруг прожектор, вздев на нос очки,
впился в спину миноносочки.
Как взревет медноголосина:
Р-р-р-астакая миноносина!
'''

words = [w.strip(punct).lower() for w in word_tokenize(poem)]
words = [w for w in words if w not in sw and w != '']

In [ ]:
snowball = SnowballStemmer("russian")

for w in words:
    print("%s: %s" % (w, snowball.stem(w)))

морям: мор
играя: игр
носится: нос
миноносцем: миноносц
миноносица: миноносиц
льнет: льнет
меду: мед
осочка: осочк
миноносцу: миноносц
миноносочка: миноносочк
конца: конц
б: б
довелось: довел
благодушью: благодуш
миноносьему: минонос
прожектор: прожектор
вздев: вздев
нос: нос
очки: очк
впился: впил
спину: спин
миноносочки: миноносочк
взревет: взревет
медноголосина: медноголосин
р-р-р-астакая: р-р-р-астак
миноносина: миноносин


#### Лемматизация

Лемматизация — процесс приведения словоформы к лемме, т.е. нормальной (словарной) форме. Это более сложная задача, чем стемминг, но и результаты дает гораздо более осмысленные, особенно для языков с богатой морфологией.

* кошке, кошку, кошкам, кошкой $\implies$ кошка
* бежал, бежит, бегу $\implies$ бежать
* белому, белым, белыми $\implies$ белый

[Pymorphy 2](http://pymorphy2.readthedocs.io/en/latest/) — полноценный морфологический анализатор, целиком написанный на Python. Для русского языка используются словари OpenCorpora и [граммемы](https://pymorphy2.readthedocs.io/en/latest/user/grammemes.html), принятые в OpenCorpora (с небольшими изменениями).

In [ ]:
%%capture
!pip install pymorphy2

In [ ]:
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()

for w in words:
    print("%s: %s :%s" % (w, snowball.stem(w), pymorphy2_analyzer.parse(w)[0].normal_form))

морям: мор :море
играя: игр :играть
носится: нос :носиться
миноносцем: миноносц :миноносец
миноносица: миноносиц :миноносица
льнет: льнет :льнуть
меду: мед :мёд
осочка: осочк :осочок
миноносцу: миноносц :миноносец
миноносочка: миноносочк :миноносочек
конца: конц :конец
б: б :б
довелось: довел :довестись
благодушью: благодуш :благодушие
миноносьему: минонос :миноносий
прожектор: прожектор :прожектор
вздев: вздев :вздеть
нос: нос :нос
очки: очк :очки
впился: впил :впиться
спину: спин :спина
миноносочки: миноносочк :миноносочек
взревет: взревет :взреветь
медноголосина: медноголосин :медноголосина
р-р-р-астакая: р-р-р-астак :р-р-р-астакать
миноносина: миноносин :миноносина


### Векторизация текста

Допустим, мы хотим применить вот такую модель для классификации текстов. Например, хотим определять спам и не спам.
$$ \hat{y} = \psi (w_1 x_1 + w_2 x_2 + ... + w_k x_k + b)$$

$\hat{y} = \lbrace spam, ham \rbrace$

$w_i$ и $b$ - это параметры, которые нужно найти в процессе обучения (настройки, fitting) модели.

А что такое для текстов $x_i$? Нам нужно получить преобразование текстов в численный вектор, с которым может работать стандартный алгоритм машинного обучения. Как это сделать?

Тут может быть много разных ответов. Самая простая идея - это 0 или 1, где 0 - если соответствующего токена нет в тексте, а 1 - если есть.

#### Мешок n-грамм

Самые мелкие структуры языка, с которыми можно работать, называются **n-граммами**.
У n-граммы есть параметр n - количество слов, которые попадают в такое представление текста.
* Если n = 1 - то смотрим на то, сколько раз каждое слово встретилось в тексте. Получаются униграммы.
* Если n = 2 - то смотрим на то, сколько раз каждая пара подряд идущих слов, встретилась в тексте. Получаются биграммы.

Численное представление текста получается путем подсчета количества n-грамм в текстах. Так как результат не зависит от порядка слов в текстах, то говорят, что такая модель представления текстов в виде векторов получается из *гипотезы представления текста как мешка слов*.

Прежде чем получать n-граммы, нужно разделить текст на токены и провести все необходимые для задачи этапы предобработки текста. Векторизация – последний этап перед построением модели.

Самый простой способ извлечь признаки из текстовых данных – использовать класс `CountVectorizer` из `sklearn`.

Объект `CountVectorizer` делает следующее:
* строит для каждого документа (каждой пришедшей ему строки) вектор размерности `n`, где `n` – количество слов или n-грам во всём корпусе;
* заполняет каждый i-тый элемент количеством вхождений слова в данный документ.

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# инициализация векторайзера с униграммами, биграммами и триграммами
vectorizer = CountVectorizer(ngram_range=(1, 3))

# fit_transform сначала обучает векторайзер, а потом сразу применает его к набору данных
vectorized_text = vectorizer.fit_transform([" ".join(tokens)])

# первые 5 объектов вектора
pd.DataFrame(vectorized_text.T.todense(), index=vectorizer.get_feature_names_out(), columns=["n-gram score"]).head()

,n-gram score
вот,3
вот дальний,1
вот дальний город,1
вот наш,1
вот наш дом,1


#### Мешок символьных n-gram

В некоторых задачах в качестве признаков могут быть использованы n-граммы символов. Для этого необходимо установить в ```CountVectorizer()``` параметр ```analyzer = 'char'```, то есть анализировать символы.

In [ ]:
# инициализация векторайзера для символов
char_vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 8))

# обучаем его и сразу применяем тексту
char_vectorized_text = char_vectorizer.fit_transform([" ".join(tokens)])

# первые 5 объектов вектора
pd.DataFrame(char_vectorized_text.T.todense(), index=char_vectorizer.get_feature_names_out(), columns=["Char n-gram score"]).head()

,Char n-gram score
в,3
во,3
вот,3
вот,3
вот д,1


Cимвольные n-граммы используются, например, для задачи определения языка. Ещё одна замечательная особенность признаков-символов – для них не нужна токенизация и лемматизация, можно использовать такой подход для языков, у которых нет готовых анализаторов.

#### TF-IDF векторизация

`TfidfVectorizer` делает то же, что и `CountVectorizer`, но в качестве значений выдает **TF-IDF** каждого слова.

**TF (term frequency)** – относительная частотность слова в документе:
$$ TF(t,d) = \frac{n_{t}}{\sum_k n_{k}} $$

**IDF (inverse document frequency)** – обратная частота документов, в которых есть это слово:
$$ IDF(t, D) = \mbox{log} \frac{|D|}{|{d : t \in d}|} $$

Перемножаем их:
$$TFIDF(t, d, D) = TF(t,d) \times IDF(i, D)$$

Идея в том, что если слово часто встречается в одном документе, но в целом по корпусу встречается в небольшом
количестве документов, у него высокий TF-IDF.

Для получения векторного представления текста с помощью TF-IDF удобно использовать `TfidfVectorizer()`, действующий, как ```CountVectorizer()```.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# инициализация векторайзера для символов
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 4))

# обучаем его и сразу применяем тексту
tfidf_vectorized_text = tfidf_vectorizer.fit_transform([" ".join(tokens)])

# первые 5 объектов вектора
pd.DataFrame(tfidf_vectorized_text.T.todense(), index=tfidf_vectorizer.get_feature_names_out(), columns=["TF-IDF"]).head()

,TF-IDF
вот,0.273861
вот дальний,0.091287
вот дальний город,0.091287
вот дальний город вот,0.091287
вот наш,0.091287


Помимо диапазона n-грамм, в каждом из трех векторайзеров можно в качестве гиперпараметра передать максимальное и минимальное число вхождения n-граммы в документ (`max_df` и `min_df`), а также максимальное количество используемых из словаря n-грамм (`max_features`). При настройке гиперпараметров векторайзеров их тоже следует учитывать.

## Задание 1

Для обработки текста выполните следующие шаги:
- токенизация;
- приведение к нижнему регистру;
- удаление стоп-слов (с расширением стандартного списка);
- очистка текста с помощью регулярных выражений;
- нормализация (лемматизация или стемминг);
- векторизация (с настройкой гиперпараметров).

Обязательными этапами предобработки являются **токенизация** и **векторизация**. Необходимо ли включать остальные шаги, нужно решить в процессе настройки модели. Порядок выполнения шагов предобработки тоже может отличаться от приведенного в списке.

Обязательно использование векторайзеров:
1. Мешок n-грамм.
2. TF-IDF.
3. Символьные n-граммы.

Для каждого из векторайзеров нужно настроить значения следующих гиперпараметров:
- `ngram_range`;
- `max_df`;
- `min_df`;
- `max_features`.

In [ ]:
# PUT YOUR CODE HERE

## Построение модели

### Наивный байесовский классификатор

Рассмотрим задачу классификации.

Есть объект $x^{(i)}$, для него нужно оценить вероятность того, что его метка класса - это $k$. Это можно записать с использованием условной вероятности.

$$P(y^{(i)}=k|x^{(i)})$$


$$P(y^{(i)}=k) != P(y^{(i)}=k|x^{(i)})$$

***Определение***
Условной вероятностью $A$ при условии $B$, называется $P(A|B)=\frac{P(AB)}{P(B)}$. Здесь $P(AB)$ — вероятность одновременного наступления событий $A$ и $B$, а $P(B)>0$.

Для оценки этой вероятности воспользуемся теоремой Байеса, которая говорит нам о том, что апостериорная вероятность - прямо пропорциональна произведению априорной вероятности на правдоподобие.

***Теорема (формула) Байеса***

$$P(A|B)=\frac{P(B|A)\,P(A)}{P(B)}$$ при $P(B) > 0$

Заметим, что если использовать формулу полной вероятности, то можно записать формулу Байеса в виде:

$$P(A_i|B)=\frac{P(B|A_i)\,P(A_i)}{\sum_{i=1}^{n}P(B|A_i)P(A_i)}$$

> Как формула Байеса связана с задачей классификации?

Пусть $X$ множество объектов, $Y$ конечное множество имён классов,
множество $X \times Y$ является вероятностным пространством с плотностью распределения $p(x,y)=p(y)p(x|y)$.
Вероятности появления объектов каждого из классов $P_y=p(y)$ называются ''априорными вероятностями классов''.
Плотности распределения $p_y(x)=p(x|y)$ называются ''функциями правдоподобия классов''.

По известным плотностям распределения $p_y(x)$ и априорным вероятностям $P_y$ всех классов $y \in Y$ строят алгоритм $a(x)$, минимизирующий вероятность ошибочной классификации.

По формуле Байеса:

$$p(y|x) = \frac{p(x|y)\,p(y)}{p(x)}$$

По формуле полной вероятности:
$$ p(x) = \sum_{y_i} (p(x|y_i)p(y_i)$$

> Откуда взять эти самые "априорные вероятности классов" и "функции правдоподобия классов"?

Априорные вероятности классов $P_y$ можно оценить согласно закону больших чисел, тогда частота появления объектов каждого из классов равна $P'_y=\frac{l_y}{l}$ где $l_y=|X^l_y|, y \in Y$ сходится по вероятности к $P_y$ при $l_y \to \infty$. Чем больше длина выборки, тем точнее выборочная оценка $P'_y$.

Функции правдоподобия нам неизвестны, на самом деле, но по данным мы можем построить их эмпирические оценки.

> Для применения наивного байесовского классификатора при работе с текстом в `sklearn` существует `MultinomialNB`.

### Оценка качества классификации

**Точность** (Accuracy) показывает долю правильных ответов классификатора от общего числа предсказаний.

**Точность** (Precision, Positive Predictive Value) отражает какой процент положительных объектов (т.е. тех, что мы считаем положительными) правильно классифицирован.

**Полнота** (Sensitivity, True Positive Rate, Recall, Hit Rate) отражает какой процент объектов положительного класса мы правильно классифицировали.

Легко построить алгоритм со 100%-й полнотой: он все объекты относит к классу 1, но при этом точность может быть очень низкой. Нетрудно построить алгоритм с близкой к 100% точностью: он относит к классу 1 только те объекты, в которых уверен, при этом полнота может быть низкая.

**F1-мера** (F1 score) является средним гармоническим точности и полноты, максимизация этого функционала приводит к одновременной максимизации этих двух «ортогональных критериев»

$$F_1 = \frac{2}{\mathrm{recall}^{-1} + \mathrm{precision}^{-1}} = 2 \cdot \frac{\mathrm{precision} \cdot \mathrm{recall}}{\mathrm{precision} + \mathrm{recall}} = \frac{\mathrm{tp}}{\mathrm{tp} + \frac12 (\mathrm{fp} + \mathrm{fn}) } $$

Также рассматривают весовое среднее гармоническое точности и полноты –  $F_\beta$-меру:

$$F_\beta = (1 + \beta^2) \cdot \frac{\mathrm{precision} \cdot \mathrm{recall}}{(\beta^2 \cdot \mathrm{precision}) + \mathrm{recall}} = \frac {(1 + \beta^2) \cdot \mathrm{tp} }{(1 + \beta^2) \cdot \mathrm{tp} + \beta^2 \cdot \mathrm{fn} + \mathrm{fp}}\,$$

Изменение $\beta$ позволяет делать один из критериев (точность или полноту) важнее при оптимизации.

> Для расчета сразу нескольких метрик качества классификации удобно воспользоваться функцией `classification_report()` из `sklearn.metrics`.

## Задание 2

По предобработанным данным постройте несколько моделей по разным векторайзерам с разными гиперпараметрами и оцените качество.

В качестве классификатора нужно использовать наивный байесовский классификатор.

Для сравнения моделей с разными векторайзерами между собой используйте метрики `precision`, `recall`, `f1-score` и `accuracy`. Для этого сформируйте таблицу (датафрейм на `pandas`), в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров. Другими словами, таблица должна иметь следующий вид:

Векторайзер и его параметры | recall | precision | f1-score | accuracy
--- | --- | --- | --- | ---
```Векторайзер 1``` |
```Векторайзер 2``` |
```Векторайзер 3``` |
... |
```Векторайзер N``` |


Лучшая модель должна работать с точностью (`accuracy`) не менее 0.74.

In [ ]:
# PUT YOUR CODE HERE

# Порядок защиты работы

**Для защиты работы необходимо:**
1. Предоставить результаты выполнения задания в виде ссылки на Google Colab.
2. Ответить на контрольные вопросы по содержанию работы.

**Примеры контрольных вопросов:**
1. Какие шаги предобработки из приведенного в задании списка не использовались в вашей работе? Почему?
2. Объясните отличие стемминга от лемматизации. Какой вид нормализации текста использовался в вашей работе?
3. Как влияет указание конкретного значения `max_features` на формирование вектора документа через `TfidfVectorizer()`?
4. В чем разница между `accuracy` и `precision`?
5. Почему для классификации текстов в этой лабораторной логично применять `MultinomialNB`, а не другую вариацию наивного байесовского классификатора из `sklearn`?

# Дополнительные материалы

1. Автоматическая обработка текстов на естественном языке и анализ данных : учеб. пособие / Большакова Е.И., Воронцов К.В., Ефремова Н.Э., Клышинский Э.С., Лукашевич Н.В., Сапин А.С. — М.: Изд-во НИУ ВШЭ, 2017. — 269 с.
1. Naive Bayes. Scikit-learn [Электронный ресурс]. URL: https://scikit-learn.org/stable/modules/naive_bayes.html#naive-bayes (дата обращения: 02.04.2024).